![VGG_Net](https://github.com/user-attachments/assets/5c13167c-9641-4e9c-a414-1219bc6fb048)


In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.utils.data as data
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [3]:
batch_size= 1
learning_rate = 0.0002
num_epoch = 100

In [4]:
img_dir = "./images"
img_data = dset.ImageFolder(img_dir, transforms.Compose([
                                    transforms.Resize(256),
                                    transforms.RandomResizedCrop(224),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor(),
            ]))
train_loader = data.DataLoader(img_data, batch_size = batch_size, shuffle = True, num_workers = 2)

- Model
  - Basic Blocks
    - 컨볼루션 연산 반복하는 부분을 함수로 구현

In [5]:
def conv_2_block(in_dim, out_dim):
    model = nn.Sequential(
        nn.Conv2d(in_dim, out_dim, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.Conv2d(out_dim, out_dim, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(2, 2)
    )
    return model

def conv_3_block(in_dim, out_dim):
    model = nn.Sequential(
        nn.Conv2d(in_dim, out_dim, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.Conv2d(out_dim, out_dim, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.Conv2d(out_dim, out_dim, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(2, 2)
    )
    return model

- VGG Model 구현

In [6]:
class VGG(nn.Module):
    def __init__(self, base_dim, num_classes = 2):
        super(VGG, self).__init__()
        self.feature = nn.Sequential(
            conv_2_block(3, base_dim),
            conv_2_block(base_dim, 2*base_dim),
            conv_3_block(2*base_dim, 4*base_dim),
            conv_3_block(4*base_dim, 8*base_dim),
            conv_3_block(8*base_dim, 8*base_dim),
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(8*base_dim * 7 * 7, 100),
            nn.ReLU(True), # inplace 연산
            nn.Linear(100, 20),
            nn.ReLU(True),
            nn.Linear(20, num_classes),
        )
    def forward(self, x):
        x = self.feature(x)
        print(x.shape)
        x = x.view(x.size(0), -1)
        print(x.shape)
        x = self.fc_layer(x)
        print(x.shape)
        return x


- 컨볼루션 연산을 하는 동안 이미지 크기 감소는 없음
- 풀링 과정에서 크기 축소

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model = VGG(base_dim = 16).to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
for i in model.named_children():
    print(i)

cpu
('feature', Sequential(
  (0): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (1): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (3): Sequenti

In [8]:
for i in range(num_epoch):
    for j, [image,label] in enumerate(train_loader):
        x = image.to(device)
        y_ = label.to(device)
        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_func(output, y_)
        loss.backward()
        optimizer.step()
    if i % 10 == 0:
        print(loss)

torch.Size([1, 128, 7, 7])
torch.Size([1, 6272])
torch.Size([1, 2])
torch.Size([1, 128, 7, 7])
torch.Size([1, 6272])
torch.Size([1, 2])
torch.Size([1, 128, 7, 7])
torch.Size([1, 6272])
torch.Size([1, 2])
tensor(0.5976, grad_fn=<NllLossBackward0>)
torch.Size([1, 128, 7, 7])
torch.Size([1, 6272])
torch.Size([1, 2])
torch.Size([1, 128, 7, 7])
torch.Size([1, 6272])
torch.Size([1, 2])
torch.Size([1, 128, 7, 7])
torch.Size([1, 6272])
torch.Size([1, 2])
torch.Size([1, 128, 7, 7])
torch.Size([1, 6272])
torch.Size([1, 2])
torch.Size([1, 128, 7, 7])
torch.Size([1, 6272])
torch.Size([1, 2])
torch.Size([1, 128, 7, 7])
torch.Size([1, 6272])
torch.Size([1, 2])
torch.Size([1, 128, 7, 7])
torch.Size([1, 6272])
torch.Size([1, 2])
torch.Size([1, 128, 7, 7])
torch.Size([1, 6272])
torch.Size([1, 2])
torch.Size([1, 128, 7, 7])
torch.Size([1, 6272])
torch.Size([1, 2])
torch.Size([1, 128, 7, 7])
torch.Size([1, 6272])
torch.Size([1, 2])
torch.Size([1, 128, 7, 7])
torch.Size([1, 6272])
torch.Size([1, 2])
torch